In [1]:
from geo_address.processing import split_address
from geo_address.processing import begin_geocode

import pandas as pd 


In [4]:
df = pd.read_csv('../../address_to_coords/data/dataset_2011_2013.csv').sample(100)

In [5]:
test_df = split_address(df, 'Addresses')

Processing row 99: 100%|███████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1746.18it/s]


**Note**: Remember to add your API Key on the text file if you're using Google V3 as in the article. If you use another API, consult the geopy documentation for the requirements. 
For example, not providing a valid API Key for GoogleV3 the following error pops up:
GeocoderQueryError: The provided API key is invalid.  

In [10]:
coords = begin_geocode(test_df)
coords

Using <geopy.geocoders.google.GoogleV3 object at 0x0000021C9EF803A0> geocoder from geopy
Preparing address field for geocoding...


Geocoding row 99 of 100: 100%|█████████████████████████████████████████████████████████████████████████| 72/72 [00:36<00:00,  1.97it/s]


[[(54.9791871, -1.6146608)],
 [(43.296482, 5.36978)],
 [(51.0246858, 3.7253383),
  (51.9845558, 5.6657696),
  (52.2660751, 6.1552165),
  (51.06783069999999, 3.7290914)],
 [(-32.8927718, 151.7041775),
  (-37.7983459, 144.960974),
  (-37.7970799, 144.9654593),
  (-42.9041118, 147.3247503),
  (-42.8295878, 147.5635746),
  (45.2089892, 5.692755)],
 [(48.790367, 2.455572),
  (48.78848230000001, 2.4567614),
  (48.697161, 2.189617),
  (45.2089892, 5.692755),
  (48.701844, 2.134078),
  (48.7964411, 2.4533704),
  (48.7965026, 2.4654091),
  (51.52508479999999, -0.1224323),
  (44.1543649, 4.647904)],
 [(50.9340991, -1.3957143),
  (48.2398824, 11.630583),
  (47.5385648, 8.22979),
  (51.574362, -1.310914)],
 [(44.8908698, 8.602508799999999)],
 [(52.2065086, 20.9790487),
  (52.2065086, 20.9790487),
  (53.5488282, 9.987170299999999),
  (39.435561, -77.43348759999999),
  (52.4083974, 16.915402)],
 [(40.4473608, -3.7384311),
  (52.08539469999999, 5.1690859),
  (51.7517617, -1.215794),
  (51.7593481, -1

In [11]:
coords


[[(54.9791871, -1.6146608)],
 [(43.296482, 5.36978)],
 [(51.0246858, 3.7253383),
  (51.9845558, 5.6657696),
  (52.2660751, 6.1552165),
  (51.06783069999999, 3.7290914)],
 [(-32.8927718, 151.7041775),
  (-37.7983459, 144.960974),
  (-37.7970799, 144.9654593),
  (-42.9041118, 147.3247503),
  (-42.8295878, 147.5635746),
  (45.2089892, 5.692755)],
 [(48.790367, 2.455572),
  (48.78848230000001, 2.4567614),
  (48.697161, 2.189617),
  (45.2089892, 5.692755),
  (48.701844, 2.134078),
  (48.7964411, 2.4533704),
  (48.7965026, 2.4654091),
  (51.52508479999999, -0.1224323),
  (44.1543649, 4.647904)],
 [(50.9340991, -1.3957143),
  (48.2398824, 11.630583),
  (47.5385648, 8.22979),
  (51.574362, -1.310914)],
 [(44.8908698, 8.602508799999999)],
 [(52.2065086, 20.9790487),
  (52.2065086, 20.9790487),
  (53.5488282, 9.987170299999999),
  (39.435561, -77.43348759999999),
  (52.4083974, 16.915402)],
 [(40.4473608, -3.7384311),
  (52.08539469999999, 5.1690859),
  (51.7517617, -1.215794),
  (51.7593481, -1